지금까지는 제공되는거 받음<br><br>
이제는 우리가 직접 단어를 벡터로 변환<br><br>
one-hot-encodeing이 아닌 dense vector

### 1. Word embedding (Word2Vec):

In [1]:
# Install once if necessary.
!pip install gensim
!pip install nltk

  Using cached https://files.pythonhosted.org/packages/09/ed/b59a2edde05b7f5755ea68648487c150c7c742361e9c8733c6d4ca005020/gensim-3.8.1-cp37-cp37m-win_amd64.whl


ERROR: gensim 3.8.1 requires smart-open>=1.8.1, which is not installed.


In [2]:
import re
import os
import nltk
import urllib
import bs4 as bs
import warnings
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
warnings.filterwarnings('ignore')
# nltk.download()

#### 1.1. Download the text data:

In [3]:
# Connect to the source.
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/Machine_learning').read()

#함수로 라이브러리 가져옴

In [4]:
# Beautiful soup object.
soup = bs.BeautifulSoup(source,'lxml') #lxml로 파싱

In [5]:
# Build a long string. 
my_text = ""
for paragraph in soup.find_all('p'):
    my_text += paragraph.text
print(my_text)

Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to perform a specific task without using explicit instructions, relying on patterns and inference instead. It is seen as a subset of artificial intelligence. Machine learning algorithms build a mathematical model based on sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to perform the task.[1][2]:2 Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or infeasible to develop a conventional algorithm for effectively performing the task.
Machine learning is closely related to computational statistics, which focuses on making predictions using computers. The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning. Data mining is a field of study within machine learning, 

#### 1.2. Preprocessing of the text data:

In [6]:
my_text = my_text.lower()  #소문자화
my_text = re.sub(r'\[[0-9]*\]',' ', my_text)
my_text = re.sub(r'\W',' ', my_text)
#my_text = re.sub(r'\s+',' ',my_text)
my_text = re.sub(r'\d+',' ',my_text)
my_text = re.sub(r'\s+',' ',my_text)

#아주 긴 단 하나의 문장이라 for문 돌릴필요 없음
#전체를 한번에 소문자화하고 전처리함

#숫자없애줌
#문자아닌거 제거
#스페이스 여러번 반복되면 하나의 스페이스로 단축

#### 1.3. Tokenization:

In [7]:
my_sentences = nltk.sent_tokenize(my_text)
my_words_0=[]
for a_sentence in my_sentences:
    my_words_0 += nltk.word_tokenize(a_sentence)
my_words_0 = [a_word for a_word in my_words_0 if len(a_word)>2 ]
my_words_0 = [a_word for a_word in my_words_0 if a_word not in stopwords.words('english')]
my_words_0 = [my_words_0]    # Required by Word2Vec. #리스트를 리스트로 에워쌈 -> Word2Vec 사용하기위해
len(my_words_0[0])

#길이 너무 짧으면 제외시킴, 3자리이상인것만 포함
#불용어 제거
#리스트로 만들어진 단어들을 또다른 리스트로 에워싸야 함

#단어의 수 3715(반복단어 있을 수 있음)

3715

#### 1.4. Train the Word2Vec model:

In [8]:
my_model = Word2Vec(my_words_0, size = 100, min_count=1)
my_words = my_model.wv.vocab
len(my_words)

#my_words_0리스트 넣음 100차원으로 줄임, 카운트가 최소 1개이상인것만
#간추려진 단어들
#키와 값으로 들어가있음 
#우리가 중요하게 생각하는건 키

1424

#### 1.5. Embedding vectors:

In [9]:
# View the dense vector corresponding to 'machine'.
my_vector = my_model.wv['machine']
print("Length = " + str(my_vector.shape[0]))
print("-"*100)
print(my_vector)

#machine에 해당하는 벡터 가져옴 (wv하면됨)
#모든 성분들이 꽉 차있음,원핫 인코딩아님 길이가 100인 벡터
#길이가 100인 machine에 해당하는 벡터가 결과로 나옴

Length = 100
----------------------------------------------------------------------------------------------------
[ 0.0011043  -0.00097925  0.00433015 -0.00257119  0.0001586   0.00409234
  0.00037186 -0.00450301 -0.00123766  0.00063321 -0.00559998  0.00305271
 -0.00356645  0.00583787 -0.00360445 -0.0040584  -0.0028743  -0.00192232
 -0.00132044  0.00116747 -0.00018445  0.00256248  0.00450314 -0.00391636
  0.00313078  0.00641195  0.00109213 -0.00489138 -0.00588081  0.00042069
 -0.00394049 -0.00090808 -0.0011267   0.00077069 -0.00070348 -0.00469152
 -0.00421611  0.00371868 -0.00496202 -0.00207414  0.00549247 -0.00561305
  0.00399379  0.00083277 -0.00199647  0.00384043 -0.00027936  0.00593427
 -0.00386305  0.00484634 -0.00429097  0.00304538  0.00408226 -0.00228252
 -0.00526288 -0.0047517  -0.00317287  0.00186569 -0.00524762 -0.00037838
 -0.00325018  0.00165739 -0.0014263   0.0011698  -0.00301538 -0.00321045
  0.0023516  -0.00042633  0.00317044 -0.00205625 -0.00299603 -0.00426426
  0.005621

#### 1.6. Most similar words:

In [10]:
my_model.wv.most_similar('learning')
#learning과 가장 유사한 단어 보여줌
#유사도 높은 순으로 나옴

[('machine', 0.3735805153846741),
 ('example', 0.36993607878685),
 ('model', 0.33721923828125),
 ('labeled', 0.33236193656921387),
 ('analysis', 0.32431983947753906),
 ('rules', 0.32063210010528564),
 ('social', 0.315327525138855),
 ('win', 0.3061886131763458),
 ('output', 0.30409398674964905),
 ('ilp', 0.30125632882118225)]

In [11]:
my_model.wv.most_similar('artificial')

[('one', 0.2998407781124115),
 ('typically', 0.29790225625038147),
 ('considerable', 0.29387372732162476),
 ('features', 0.28923559188842773),
 ('sparse', 0.2888709306716919),
 ('semi', 0.2851007282733917),
 ('characteristics', 0.2836957573890686),
 ('represented', 0.28068724274635315),
 ('output', 0.27863818407058716),
 ('classification', 0.2735111117362976)]

In [ ]:
# Operation:
# global - cooling + warming = ???
my_model.wv.most_similar(positive=['machine','human'], negative= ['learning'])

#machine에 human더하고 learning 뺌
#벡터연산해서 가장 가까운 벡터는 무엇인가? 
#resposibility

### 2. Using a pre-trained model from Google: 이부분은 다운로드해야 함..

Download ["GoogleNews-vectors-negative300.bin"](https://docs.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM&export=download) and uncompress. <br>
**Caution**: Size compressed ~ 1.6 Gb, uncompressed ~ 3.5 Gb.

In [2]:
# Go to the directory where the downloaded file is located. 
# os.chdir(r'~~')                # Please, replace the path with your own.   

In [ ]:
# Load the file.
filename = "GoogleNews-vectors-negative300.bin"
a_model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [ ]:
# The most similar words to 'king' or 'kings'.
a_model.most_similar(['king','kings'])

In [ ]:
# Operation: queen(queens) - woman(women) + man(men) = ???
a_model.most_similar(positive=['queen','queens','man','men'], negative= ['woman','women'])